In [11]:
import glob
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from prody import *
import openpyxl

In [2]:
int_en_files = glob.glob('../*/*/EC_Data/*/energies_intEnTotal.csv')

In [3]:
df_metadata = pd.DataFrame(columns=['subtype','peptide','filename'])

i = 0
for fl in int_en_files:
    matches = re.search('.*EC_Data/(HLA_.*)_(.*)/energies_.*', fl)
    if matches:
        subtype_name = matches.groups()[0]
        peptide = matches.groups()[1]
        df_metadata.loc[i,'filename'] = fl
        df_metadata.loc[i,'subtype'] = subtype_name
        df_metadata.loc[i,'peptide'] = peptide
        i += 1

In [4]:
df_ie_list = list()

for row in df_metadata.iterrows():
    df = pd.read_csv(row[1]['filename'])
    syst = parsePDB('/'.join(row[1]['filename'].split('/')[:-1])+'/system.pdb')
    df = df.transpose()

    for row_df in df.iterrows():
        if row_df[0].startswith('Unnamed: '):
            continue
        pair = row_df[0].split('-')
        res1 = int(pair[0])
        res2 = int(pair[1])
        df.loc[row_df[0],'res1_index'] = res1
        df.loc[row_df[0],'res2_index'] = res2
        chid1 = syst.select(f'resindex {res1}').getChids()[0]
        chid2 = syst.select(f'resindex {res2}').getChids()[0]
        resnum1 = syst.select(f'resindex {res1}').getResnums()[0]
        resnum2 = syst.select(f'resindex {res2}').getResnums()[0]
        df.loc[row_df[0],'res1_chid'] = chid1
        df.loc[row_df[0],'res2_chid'] = chid2
        df.loc[row_df[0],'res1_resnum'] = resnum1
        df.loc[row_df[0],'res2_resnum'] = resnum2

    df.dropna(inplace=True)
    df.reset_index(inplace=True)
    df.rename(columns={0: 'IE'}, inplace=True)
    df.drop(['index'], axis=1, inplace=True)
    df['subtype'] = row[1]['subtype']
    df['peptide'] = row[1]['peptide']

    df_ie_list.append(df)
    
df_ie_master = pd.concat(df_ie_list,axis=0, ignore_index = False)

@> 6149 atoms and 1 coordinate set(s) were parsed in 0.15s.
/tmp/ipykernel_8464/3312471821.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'C' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[row_df[0],'res1_chid'] = chid1
/tmp/ipykernel_8464/3312471821.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'C' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[row_df[0],'res2_chid'] = chid2
@> 6122 atoms and 1 coordinate set(s) were parsed in 0.15s.
/tmp/ipykernel_8464/3312471821.py:20: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value 'C' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.loc[row_df[0],'res1_chid'] = chid1
/tmp/ipykernel_8

In [7]:
df_ie_master

,IE,res1_index,res2_index,res1_chid,res2_chid,res1_resnum,res2_resnum,subtype,peptide
0,-32.727512,384.0,383.0,C,C,9.0,8.0,HLA_A_02_01,AAAWYLWEV
1,-37.339195,376.0,377.0,C,C,1.0,2.0,HLA_A_02_01,AAAWYLWEV
2,-15.444854,65.0,377.0,A,C,66.0,2.0,HLA_A_02_01,AAAWYLWEV
3,-0.489061,376.0,161.0,C,A,1.0,162.0,HLA_A_02_01,AAAWYLWEV
4,-0.085346,384.0,124.0,C,A,9.0,125.0,HLA_A_02_01,AAAWYLWEV
...,...,...,...,...,...,...,...,...,...
251,-20.213072,377.0,150.0,C,A,7.0,151.0,HLA_C_14_02,YYKKTFSAL
252,0.029165,373.0,22.0,C,A,3.0,23.0,HLA_C_14_02,YYKKTFSAL
253,-0.245149,158.0,375.0,A,C,159.0,5.0,HLA_C_14_02,YYKKTFSAL
254,-0.047932,377.0,122.0,C,A,7.0,123.0,HLA_C_14_02,YYKKTFSAL


In [12]:
df_ie_master.to_excel("df_ie_master.xlsx") 